In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/home/philippe/MolGenDocking


/home/philippe/miniconda3/envs/MolGen/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
from notebooks.utils import *

import pandas as pd

In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt


def plot_all_cols(df, N_COLS, fig, escape=["smiles", "pdb_id"]):
    outer_grid = fig.add_gridspec(N_COLS, 1, wspace=0.1, hspace=0.5)


    i = 0
    j = 0
    n_cols = (df[[c for c in df.columns if not c in escape]].shape[1]-1) // N_COLS +1

    grid = outer_grid[0].subgridspec(1, ncols=n_cols)

    for col in df.columns:
        if col not in escape:
            ax = plt.subplot(grid[i])
            sns.histplot(df[col], bins=100, ax=ax)
            ax.set_title(col)
            i += 1
            if i == n_cols and not j == N_COLS - 1:
                i = 0
                j += 1
                if j == N_COLS - 1:
                    grid = outer_grid[j].subgridspec(1, ncols=n_cols + (df.shape[1]-1) % N_COLS)
                else:
                    grid = outer_grid[j].subgridspec(1, ncols=n_cols)

fig = plt.figure(figsize=(20, 10))
plot_all_cols(df, 4, fig, escape=["smiles", "pdb_id"])

In [ ]:
DATA_PATH = "data/mol_orz"

target_info_df = pd.read_csv(f"{DATA_PATH}/pockets_info.csv").drop(columns=["center", "size"])
target_info_df["pocket score"] = target_info_df["pocket score"].clip(0,1)
fig = plt.figure(figsize=(20, 7))
plot_all_cols(target_info_df, 3, fig)

In [ ]:
target_info_df[(target_info_df["drug score"] >0.5) & (target_info_df["pocket score"] >0.5)]